In [37]:
import os
import numpy as np
from sklearn.svm import SVC
from micromlgen import port


In [38]:


def load_features(dataset_folder):
#   Load features and labels from the dataset folder.
    #The first row of each CSV file contains 'SSID' and 'RSSI'.
   # """
    features = []
    labels = []
    class_map = {}
    class_id = 0

    for file in os.listdir(dataset_folder):
        if file.endswith(".csv"):
            file_path = os.path.join(dataset_folder, file)
            data = np.genfromtxt(file_path, delimiter=',', skip_header=1, dtype=None, encoding=None)

            # Process each row in the CSV file
            for entry in data:
                ssid = entry[0]  # First column: SSID
                try:
                # Convert valid RSSI values to integers
                    rssi = int(entry[1]) if entry[1] != '-' else 0  # Default to 0 if missing or invalid
                except ValueError:
                    rssi = 0  # Default to 0 if conversion fails

                # Map SSIDs to numeric class IDs
                if ssid not in class_map:
                    class_map[ssid] = class_id 
                    # SSID as key, class_id as value
                    class_id += 1

                # Append the RSSI value as a feature and the corresponding label
                features.append([rssi])  # Single RSSI value wrapped in a list
                labels.append(class_map[ssid])  # Numeric label corresponding to SSID

    return np.array(features), np.array(labels), class_map


In [39]:
# Load dataset
dataset_folder = 'C:/Users/Administrator/Downloads'  # Replace with the path to your dataset folder
X, y, class_map = load_features(dataset_folder)

In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Load dataset
dataset_folder = 'C:/Users/Administrator/Downloads'  # Replace with the path to your dataset folder

# Load features and labels from your custom function
X, y, class_map = load_features(dataset_folder)

# Debugging step: Ensure proper data shapes
print(f"Feature shape: {X.shape}, Label shape: {y.shape}")
print(f"Unique Labels (Classes): {np.unique(y)}")

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model on the training data
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Test the model on the test set
y_pred = classifier.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Display evaluation metrics
print("Evaluation Metrics on Test Set:")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)

# Invert the class_map
inverse_class_map = {v: k for k, v in class_map.items()}

# Use the inverted class map for target names
target_names = [inverse_class_map[key] for key in sorted(np.unique(y))]
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))



#print("\nClassification Report:")
#print(classification_report(y_test, y_pred, target_names = [class_map[key] for key in sorted(np.unique(y))]))

# Create an inverse map for numeric-to-classname conversion
inverse_class_map = {v: k for k, v in class_map.items()}

# Testing the model with new data
print("\nTesting with new data:")
new_data = np.array([[-50], [-40], [-30]])  # Example RSSI values to test
predicted_labels = classifier.predict(new_data)

for rssi, label in zip(new_data.flatten(), predicted_labels):
    print(f"RSSI: {rssi} -> Predicted Class: {inverse_class_map[label]}")

# Export model as Arduino-compatible C code
try:
    c_code = port(classifier, classmap=class_map)
except AttributeError as e:
    print(f"Error in exporting the model: {e}")
    raise

# Save the model to the specified path
saved_path = 'C:/Users/Administrator/Desktop/models'  # Your path
model_name = 'model_logistic.h'
output_file = os.path.join(saved_path, model_name)

with open(output_file, 'w') as f:
    f.write(c_code)

print(f"Model exported and saved to: {output_file}")


Feature shape: (2645, 1), Label shape: (2645,)
Unique Labels (Classes): [0 1 2]
Evaluation Metrics on Test Set:
Accuracy: 100.00%
Precision: 1.00
Recall: 1.00
F1-Score: 1.00

Confusion Matrix:
[[178   0   0]
 [  0 172   0]
 [  0   0 179]]

Classification Report:
              precision    recall  f1-score   support

   Inturai_3       1.00      1.00      1.00       178
   Inturai_2       1.00      1.00      1.00       172
   Inturai_1       1.00      1.00      1.00       179

    accuracy                           1.00       529
   macro avg       1.00      1.00      1.00       529
weighted avg       1.00      1.00      1.00       529


Testing with new data:
RSSI: -50 -> Predicted Class: Inturai_2
RSSI: -40 -> Predicted Class: Inturai_2
RSSI: -30 -> Predicted Class: Inturai_3
Model exported and saved to: C:/Users/Administrator/Desktop/models\model_logistic.h
